<a href="https://colab.research.google.com/github/AArna1211/Traffic-prediction/blob/main/TP_VMD%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install vmdpy

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from vmdpy import VMD
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [23]:
data = pd.read_csv('/content/Traffic.csv')

In [24]:
data.shape

(2976, 9)

In [25]:
data.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal


In [26]:
data.isna().sum()

,0
Time,0
Date,0
Day of the week,0
CarCount,0
BikeCount,0
BusCount,0
TruckCount,0
Total,0
Traffic Situation,0


In [27]:
# Convert Time & Date columns to datetime format
data["Timestamp"] = pd.to_datetime(data["Time"] + " " + data["Date"].astype(str))
data = data.sort_values(by="Timestamp")


<ipython-input-27-cb43e09a6d1f>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Timestamp"] = pd.to_datetime(data["Time"] + " " + data["Date"].astype(str))


In [29]:
# Encode categorical "Traffic Situation"
label_encoder = LabelEncoder()
data["Traffic Situation Encoded"] = label_encoder.fit_transform(data["Traffic Situation"])

In [30]:
data.shape

(2976, 11)

In [31]:
# Select features for decomposition
features = ["CarCount", "BikeCount", "BusCount", "TruckCount", "Total"]
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[features])


In [32]:
# Apply VMD to decompose "Total" traffic into intrinsic mode functions (IMFs)
alpha = 2000  # Moderate bandwidth constraint
tau = 0        # Noise-tolerance (set to 0 for noise-free)
K = 3          # Number of modes (adjust based on data complexity)
DC = 0         # No DC component constraint
init = 1       # Initialize spectrum uniformly
Tol = 1e-6     # Convergence tolerance

In [33]:
u, _, _ = VMD(data_scaled[:, -1], alpha, tau, K, DC, init, Tol)  # Decomposing "Total"


In [34]:
# Prepare LSTM input (reshape and normalize IMFs)
u_scaled = [scaler.fit_transform(imf.reshape(-1, 1)).flatten() for imf in u]
u_scaled = np.array(u_scaled).T  # Transpose to match (samples, features)

In [35]:
# Create time-series sequences
def create_sequences(data, labels, seq_length=2):  # Adjusted for small dataset
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(labels[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 2
X, y = create_sequences(u_scaled, data["Traffic Situation Encoded"].values, seq_length)
X = X.reshape(X.shape[0], X.shape[1], K)  # Reshape for LSTM

In [36]:
# Split into train and test sets
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, K)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [43]:
# Train model
epochs = 20
batch_size = 1  # Adjusted for small dataset
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


Epoch 1/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7428 - loss: 0.6934 - val_accuracy: 0.7529 - val_loss: 0.6760
Epoch 2/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7267 - loss: 0.7060 - val_accuracy: 0.7395 - val_loss: 0.6908
Epoch 3/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7314 - loss: 0.7052 - val_accuracy: 0.7563 - val_loss: 0.6664
Epoch 4/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7462 - loss: 0.6791 - val_accuracy: 0.7513 - val_loss: 0.6878
Epoch 5/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7581 - loss: 0.6569 - val_accuracy: 0.7412 - val_loss: 0.6698
Epoch 6/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7224 - loss: 0.7000 - val_accuracy: 0.7429 - val_loss: 0.6638
Epoch 7/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7175 - loss: 0.7140 - val_accuracy: 0.7513 - val_loss: 0.6629
Epoch 8/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7487 - loss: 0.6742

In [44]:
# Predict on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Decode labels
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_test_labels = label_encoder.inverse_transform(y_test)

# Evaluate model
accuracy = np.mean(y_pred_classes == y_test)
print(f"Test Accuracy: {accuracy:.4f}")


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Test Accuracy: 0.7613
